In [42]:
import squigglepy as sq
import numpy as np
import pandas as pd
from squigglepy.numbers import K, M

sq.set_seed(42)
np.random.seed(42)
N_SAMPLES = 10_000

## Trainium2 Quarterly Production Timeline

Amazon disclosed in its [Q4 2025 earnings](https://s2.q4cdn.com/299287126/files/doc_earnings/2025/q4/earnings-result/AMZN-Q4-2025-Earnings-Release.pdf) (February 5, 2026) that "Trainium2 is fully subscribed with 1.4 million chips landed." This is our primary anchor for cumulative stock. "Landed" likely means operational; possible time lags modeled as ~10% uncertainty in either direction.

### Distributing production over time

[SemiAnalysis](https://newsletter.semianalysis.com/p/amazons-ai-resurgence-aws-anthropics-multi-gigawatt-trainium-expansion) estimated relative quarterly Trainium2 package production, which implies:

- **2024 was ~20% of the cumulative total** (total 2025 production ~4x higher than 2024). We assume a range of 15-30%.
- **2024 quarterly growth was ~2.5x** per quarter (digitized from the SemiAnalysis chart). We model this as 2–3x.
- **2025 H1 was 40–45% of 2025 production**, implying moderate within-year growth (~11–22%/quarter).
- For simplicity we assume that within each year, production grew smoothly and exponentially in each quarter. 

This yields a rough timeline of ~300k Trainium2 in 2024, ~450k in 2025 H1, ~650k in 2025 H2.

In [49]:
# ==============================================
# MODEL PARAMETERS
# ==============================================
# Two-phase model for Trainium2 quarterly production over Q1 2024 – Q4 2025.
#
# Within each year, quarterly production grows exponentially:
#   q_i = a * r^(i-1),  i = 1..4
#   Total constraint: sum(q_i) = T_year  =>  a = T_year * (r-1) / (r^4 - 1)
#
# The Q4 2024 → Q1 2025 transition is unconstrained (discontinuous).
#
# For an exponential series over 4 quarters:
#   H1 share = (q1 + q2) / total = 1 / (1 + r²)
#   So r = sqrt(1/h1_share - 1)

QUARTERS = [
    'Q1 2024', 'Q2 2024', 'Q3 2024', 'Q4 2024',
    'Q1 2025', 'Q2 2025', 'Q3 2025', 'Q4 2025',
]

# Total Trainium2 stock produced over 8 quarters: 1.4M +/- 10%
total_trainium2_stock = sq.norm(1.26 * M, 1.54 * M)  # 90% CI

# 2024 share of cumulative total (90% CI)
share_of_total_produced_in_2024 = sq.norm(0.15, 0.30, lclip=0.01, rclip=0.99)

# 2025 H1 share of 2025 total → determines 2025 quarterly growth rate
h1_share_of_2025_production = sq.norm(0.40, 0.45, lclip=0.01, rclip=0.99)

# 2024 within-year quarterly growth rate
# SemiAnalysis estimates ~2.5x quarterly growth; we use a 90% CI of 2–3x
quarterly_growth_rate_2024 = sq.to(2.0, 3.0)

# Check implied 2025 growth rate
h1_share_samples = h1_share_of_2025_production @ N_SAMPLES
implied_quarterly_growth_2025 = np.sqrt(1.0 / h1_share_samples - 1.0)
print("Implied 2025 quarterly growth rate:")
print(f"  Median: {np.median(implied_quarterly_growth_2025):.3f}")
print(f"  90% CI: [{np.percentile(implied_quarterly_growth_2025, 5):.3f}, {np.percentile(implied_quarterly_growth_2025, 95):.3f}]")
print()
print("2024 quarterly growth rate (prior):")
growth_2024_samples = quarterly_growth_rate_2024 @ N_SAMPLES
print(f"  Median: {np.median(growth_2024_samples):.3f}")
print(f"  90% CI: [{np.percentile(growth_2024_samples, 5):.3f}, {np.percentile(growth_2024_samples, 95):.3f}]")

Implied 2025 quarterly growth rate:
  Median: 1.163
  90% CI: [1.105, 1.225]

2024 quarterly growth rate (prior):
  Median: 2.441
  90% CI: [1.997, 2.991]


In [50]:
# ==============================================
# MONTE CARLO SIMULATION
# ==============================================

quarterly_production = {q: np.zeros(N_SAMPLES) for q in QUARTERS}

for s in range(N_SAMPLES):
    total_stock = total_trainium2_stock @ 1
    frac_2024 = share_of_total_produced_in_2024 @ 1

    production_2024_total = total_stock * frac_2024
    production_2025_total = total_stock - production_2024_total

    # 2025: derive growth rate from H1 share constraint
    h1_share = h1_share_of_2025_production @ 1
    growth_2025 = np.sqrt(1.0 / h1_share - 1.0)
    if abs(growth_2025 - 1) < 1e-9:
        base_2025 = production_2025_total / 4
    else:
        base_2025 = production_2025_total * (growth_2025 - 1) / (growth_2025**4 - 1)

    # 2024: sample growth rate from prior
    growth_2024 = quarterly_growth_rate_2024 @ 1
    if abs(growth_2024 - 1) < 1e-9:
        base_2024 = production_2024_total / 4
    else:
        base_2024 = production_2024_total * (growth_2024 - 1) / (growth_2024**4 - 1)

    for i in range(4):
        quarterly_production[QUARTERS[i]][s] = base_2024 * growth_2024**i
        quarterly_production[QUARTERS[4 + i]][s] = base_2025 * growth_2025**i

print("Simulation complete.")

Simulation complete.


In [51]:
# ==============================================
# QUARTERLY PRODUCTION SUMMARY
# ==============================================

print("Trainium2 Quarterly Production Estimates")
print(f"{'Quarter':<12} {'p5':>12} {'Median':>12} {'p95':>12}")
print("-" * 50)

for q in QUARTERS:
    arr = quarterly_production[q]
    print(f"{q:<12} {int(np.percentile(arr, 5)):>12,} {int(np.percentile(arr, 50)):>12,} {int(np.percentile(arr, 95)):>12,}")

total = sum(quarterly_production[q] for q in QUARTERS)
print("-" * 50)
print(f"{'TOTAL':<12} {int(np.percentile(total, 5)):>12,} {int(np.percentile(total, 50)):>12,} {int(np.percentile(total, 95)):>12,}")

# Sanity checks
total_2024 = sum(quarterly_production[q] for q in QUARTERS[:4])
total_2025 = sum(quarterly_production[q] for q in QUARTERS[4:])
h1_2025 = quarterly_production['Q1 2025'] + quarterly_production['Q2 2025']
print()
print("Sanity checks:")
print(f"  2024 share: {np.median(total_2024 / total):.1%}  (target: 15-30%)")
print(f"  2025 H1 share of 2025: {np.median(h1_2025 / total_2025):.1%}  (target: 40-45%)")
print(f"  Q4 2024 median: {int(np.median(quarterly_production['Q4 2024'])):,}")
print(f"  Q1 2025 median: {int(np.median(quarterly_production['Q1 2025'])):,}")
print(f"  Jump ratio Q1 2025/Q4 2024: {np.median(quarterly_production['Q1 2025'] / quarterly_production['Q4 2024']):.2f}x")

Trainium2 Quarterly Production Estimates
Quarter                p5       Median          p95
--------------------------------------------------
Q1 2024             6,743       12,858       23,176
Q2 2024            19,060       31,502       48,625
Q3 2024            50,527       77,321      107,169
Q4 2024           123,139      188,975      264,045
Q1 2025           179,592      212,617      250,086
Q2 2025           213,307      247,436      285,215
Q3 2025           249,316      287,852      330,480
Q4 2025           286,911      334,936      390,528
--------------------------------------------------
TOTAL           1,261,129    1,399,897    1,543,221

Sanity checks:
  2024 share: 22.4%  (target: 15-30%)
  2025 H1 share of 2025: 42.5%  (target: 40-45%)
  Q4 2024 median: 188,975
  Q1 2025 median: 212,617
  Jump ratio Q1 2025/Q4 2024: 1.12x


In [52]:
# ==============================================
# QUARTERLY PRODUCTION (ROUNDED TO NEAREST 1K)
# ==============================================

def round_k(x):
    return int(round(x / 5000) * 5000)

print("Trainium2 Quarterly Production (rounded to nearest 1K)")
print(f"{'Quarter':<12} {'p5':>12} {'Median':>12} {'p95':>12}")
print("-" * 50)

for q in QUARTERS:
    arr = quarterly_production[q]
    print(f"{q:<12} {round_k(np.percentile(arr, 5)):>12,} {round_k(np.percentile(arr, 50)):>12,} {round_k(np.percentile(arr, 95)):>12,}")

total = sum(quarterly_production[q] for q in QUARTERS)
print("-" * 50)
print(f"{'TOTAL':<12} {round_k(np.percentile(total, 5)):>12,} {round_k(np.percentile(total, 50)):>12,} {round_k(np.percentile(total, 95)):>12,}")

Trainium2 Quarterly Production (rounded to nearest 1K)
Quarter                p5       Median          p95
--------------------------------------------------
Q1 2024             5,000       15,000       25,000
Q2 2024            20,000       30,000       50,000
Q3 2024            50,000       75,000      105,000
Q4 2024           125,000      190,000      265,000
Q1 2025           180,000      215,000      250,000
Q2 2025           215,000      245,000      285,000
Q3 2025           250,000      290,000      330,000
Q4 2025           285,000      335,000      390,000
--------------------------------------------------
TOTAL           1,260,000    1,400,000    1,545,000


In [53]:
# ==============================================
# CUMULATIVE RUNNING TOTALS
# ==============================================

print("Trainium2 Cumulative Production")
print(f"{'Quarter':<12} {'p5':>12} {'Median':>12} {'p95':>12}")
print("-" * 50)

running = np.zeros(N_SAMPLES)
cumulative_samples = {}
for q in QUARTERS:
    running = running + quarterly_production[q]
    cumulative_samples[q] = running.copy()
    print(f"{q:<12} {int(np.percentile(running, 5)):>12,} {int(np.percentile(running, 50)):>12,} {int(np.percentile(running, 95)):>12,}")

Trainium2 Cumulative Production
Quarter                p5       Median          p95
--------------------------------------------------
Q1 2024             6,743       12,858       23,176
Q2 2024            25,939       44,358       71,539
Q3 2024            77,732      122,032      176,739
Q4 2024           206,552      312,744      427,474
Q1 2025           429,362      526,706      632,000
Q2 2025           673,778      774,358      880,194
Q3 2025           949,879    1,063,990    1,181,395
Q4 2025         1,261,129    1,399,897    1,543,221


In [54]:
# ==============================================
# CSV EXPORT: QUARTERLY CHIP TIMELINES
# ==============================================

from datetime import datetime

timestamp = datetime.now().strftime("%m-%d-%Y %H:%M")
generated_note = f"Estimates generated on: {timestamp}"

def get_quarter_dates(quarter):
    """Return (start_date, end_date) for a calendar quarter like 'Q1 2024'."""
    q_num = int(quarter[1])
    year = int(quarter.split()[1])
    dates = {
        1: (f"1/1/{year}", f"3/31/{year}"),
        2: (f"4/1/{year}", f"6/30/{year}"),
        3: (f"7/1/{year}", f"9/30/{year}"),
        4: (f"10/1/{year}", f"12/31/{year}"),
    }
    return dates[q_num]

rows = []
for q in QUARTERS:
    arr = quarterly_production[q]
    start_date, end_date = get_quarter_dates(q)

    rows.append({
        'Name': f"{q} - Trainium2",
        'Chip manufacturer': 'Amazon',
        'Start date': start_date,
        'End date': end_date,
        'Compute estimate in H100e (median)': '',
        'H100e (5th percentile)': '',
        'H100e (95th percentile)': '',
        'Number of Units': int(np.percentile(arr, 50)),
        'Number of Units (5th percentile)': int(np.percentile(arr, 5)),
        'Number of Units (95th percentile)': int(np.percentile(arr, 95)),
        'Source / Link': '',
        'Notes': generated_note,
        'Chip type': 'Trainium2',
        'Last Modified By': '',
        'Last Modified': '',
    })

df = pd.DataFrame(rows)
df.to_csv('csv_export/trainium_chip_timelines.csv', index=False)
print(f"Exported {len(df)} rows to csv_export/trainium_chip_timelines.csv")
print(df[['Name', 'Number of Units (5th percentile)', 'Number of Units', 'Number of Units (95th percentile)']].to_string(index=False))

Exported 8 rows to csv_export/trainium_chip_timelines.csv
               Name  Number of Units (5th percentile)  Number of Units  Number of Units (95th percentile)
Q1 2024 - Trainium2                              6743            12858                              23176
Q2 2024 - Trainium2                             19060            31502                              48625
Q3 2024 - Trainium2                             50527            77321                             107169
Q4 2024 - Trainium2                            123139           188975                             264045
Q1 2025 - Trainium2                            179592           212617                             250086
Q2 2025 - Trainium2                            213307           247436                             285215
Q3 2025 - Trainium2                            249316           287852                             330480
Q4 2025 - Trainium2                            286911           334936                        